# Exercises-MRI-segmentation

Coding exercises for appying to the position at the Paris Brain Institute. The base code is taken from the following tutorial: https://colab.research.google.com/github/fepegar/torchio-notebooks/blob/main/notebooks/TorchIO_MONAI_PyTorch_Lightning.ipynb#scrollTo=QixbF3koO99H. 

TODO: write an introduction to the problem, the type of data that is going to be used, the number of labels, training strategy, etc...

# Exercise 1
- Write a training code for a similar training as in the tutorial, but without the
pytorch_lightning library.
- Make one script with a command line for training.
- In the training loop use the automatic mixed precision from Pytorch (with autocast and
GradScaler) in order to train with FP16 precision instead of the default FP32.

In [ ]:
# extra imports
import numpy as np
import os
from tqdm import tqdm
import torch
import monai
from utils.plotting import plot_image_label
from utils.training_funcs import training_loop
from utils.MedicalDecathlonClass import MedicalDecathlonDataModule

%load_ext autoreload
%autoreload 2

# hyperparameters
config = {
    "task": "Task04_Hippocampus",
    "google_id": "1RzPB1_bqzQhlWvU-YGvZzhx2omcDh38C",
    "batch_size": 16,
    "train_val_ratio": 0.8,
    "epochs": 100,
    "lr": 1e-2,
    "early_stopping": 10,  # -1 to disable, else insert patience
    "mixed_precision": False,
    "Nit": 8,  # number of iterations for the training loop, None if you want to do all
    "train_from_checkpoint": None,  # None to train from scratch, else insert path to the model weights
    "fine_tune": False,  # True to fine-tune the model, else False to train from scratch
    "best_models_dir": "best_models",
}

assert (
    config["early_stopping"] == -1 or config["early_stopping"] > 0
), "early_stopping must be -1 or > 0"
assert (
    config["train_val_ratio"] > 0 and config["train_val_ratio"] < 1
), "train_val_ratio must be > 0 and < 1"
assert config["batch_size"] > 0, "batch_size must be > 0"

In [ ]:
# data download & preparation
data = MedicalDecathlonDataModule(
    task=config["task"],
    google_id=config["google_id"],
    batch_size=config["batch_size"],
    train_val_ratio=config["train_val_ratio"],
)

data.prepare_data()
data.setup()

train_data_loader = data.train_dataloader()
val_data_loader = data.val_dataloader()
test_data_loader = data.test_dataloader()

In [ ]:
# visualize a training example
batch = next(iter(train_data_loader))

batch_image = batch["image"]["data"]
batch_label = batch["label"]["data"]

print(f"The shape of the data is {batch_image.shape}")

plot_image_label(batch_image, batch_label, slice_idx=30)

# Define the model

In [ ]:
# U-Net model from monai
model = monai.networks.nets.UNet(
    dimensions=3,
    in_channels=1,
    out_channels=3,
    channels=(8, 16, 32, 64),
    strides=(2, 2, 2),
)

criterion = monai.losses.DiceCELoss(softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training Loop


In [ ]:
training_loop(
    train_data_loader=train_data_loader,
    val_data_loader=val_data_loader,
    device=device,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=config["epochs"],
    early_stopping=config["early_stopping"],
    train_from_checkpoint=config["train_from_checkpoint"],
    fine_tune=config["fine_tune"],
    best_models_dir=config["best_models_dir"],
    mixed_precision=config["mixed_precision"],
    Nit=config["Nit"],
)

# Fine-tuning thoughts

To understand how to fine tune the model, I decided to print the model structure and shapes of the parameters it is made of. Then I saw that the last 2 sets of parameters corresponded to the last ConvTranspose3d and the corresponding bias, with shapes `torch.Size([16, 3, 3, 3, 3])` and `torch.Size([3])`. 

Hence I decided to freeze all the other weights and only keep thos for the fine-tuning.

In [ ]:
model

In [ ]:
for param in list(model.parameters()):
    print(param.shape)